In [53]:
from torch_geometric.data import HeteroData
import torch
from TwibotSmallEdgeHetero import TwibotSmallEdgeHetero


embedding_size,dropout,lr,weight_decay,svdComponents=128,0.3,1e-3,5e-3,100

In [54]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
homogeneous = TwibotSmallEdgeHetero(device=device,process=True,save=True,dev=False, svdComponents=svdComponents)

des_tensor,tweets_tensor,num_prop,category_prop,edge_index,edge_type,labels,train_idx,val_idx,test_idx=homogeneous.dataloader()

Loading train.json
Loading test.json
Small dataset version, not loading support.json
Loading dev.json
Finished
Loading labels...   Finished
Loading user description embeddings
Finished
Running tweet embedding
Finished
Processing feature3...   Finished
Processing feature4...   Finished
Building graph   Finished


In [55]:
# print(.shape)
tweeterEdgeIndices = (edge_type == 4).nonzero().squeeze()
print(tweeterEdgeIndices.shape)

tweeterEdges = edge_index[:,tweeterEdgeIndices]
print(tweeterEdges)
print(tweeterEdges.shape)

torch.Size([1999788])
tensor([[      0,       0,       0,  ...,   11825,   11825,   11825],
        [  11826,   11827,   11828,  ..., 2011611, 2011612, 2011613]])
torch.Size([2, 1999788])


In [56]:
## initialising the dataset object for our heterogeneous dataset

data = HeteroData()
data['user'].des = des_tensor
data['user'].cat = category_prop
data['user'].num = num_prop
# data['user'].x = None
data['user'].x = torch.cat((des_tensor, category_prop, num_prop), dim=1)
data['user'].y = labels
data['user'].train_idx = train_idx
data['user'].val_idx = val_idx
data['user'].test_idx = test_idx
data['tweet'].x = tweets_tensor

data['user', 'following', 'user'].edge_index = edge_index[:, (edge_type == 0).nonzero().squeeze()]
data['user', 'followedBy', 'user'].edge_index = edge_index[:, (edge_type == 1).nonzero().squeeze()]
data['tweet', 'mentions', 'user'].edge_index = edge_index[:, (edge_type == 2).nonzero().squeeze()]
data['tweet', 'retweets', 'user'].edge_index = edge_index[:, (edge_type == 3).nonzero().squeeze()]
data['user','writes', 'tweet'].edge_index = edge_index[:, (edge_type == 4).nonzero().squeeze()]

In [57]:
print(data.has_isolated_nodes())
print(data.has_self_loops())
print(data.is_undirected())





True
False
False


In [58]:


class HeteroTwibot():
    def __init__(self, edgeHetero: TwibotSmallEdgeHetero):
        des_tensor,tweets_tensor,num_prop,category_prop,edge_index,edge_type,labels,train_idx,val_idx,test_idx=edgeHetero.dataloader()
        self.data = HeteroData()

        self.data['user'].des = des_tensor
        self.data['user'].cat = category_prop
        self.data['user'].num = num_prop
        # self.data['user'].x = None
        self.data['user'].x = torch.cat((des_tensor, category_prop, num_prop), dim=1)
        self.data['user'].y = labels
        self.data['user'].train_idx = train_idx
        self.data['user'].val_idx = val_idx
        self.data['user'].test_idx = test_idx
        self.data['tweet'].x = tweets_tensor

        self.data['user', 'following', 'user'].edge_index = edge_index[:, (edge_type == 0).nonzero().squeeze()]
        self.data['user', 'followedBy', 'user'].edge_index = edge_index[:, (edge_type == 1).nonzero().squeeze()]
        self.data['tweet', 'mentions', 'user'].edge_index = edge_index[:, (edge_type == 2).nonzero().squeeze()]
        self.data['tweet', 'retweets', 'user'].edge_index = edge_index[:, (edge_type == 3).nonzero().squeeze()]
        self.data['user','writes', 'tweet'].edge_index = edge_index[:, (edge_type == 4).nonzero().squeeze()]
        

In [59]:
heteroTwibot = HeteroTwibot(homogeneous)

print(heteroTwibot.data.has_isolated_nodes())
print(heteroTwibot.data.has_self_loops())
print(heteroTwibot.data.is_undirected())

Loading labels...   Finished
Loading user description embeddings
Finished
Running tweet embedding
Finished
Processing feature3...   Finished
Processing feature4...   Finished
Building graph   Finished
True
False
False


In [60]:
from HeteroTwibot import initializeHeteroTwibot

dataset = initializeHeteroTwibot(homogeneous)

Loading labels...   Finished
Loading user description embeddings
Finished
Running tweet embedding
Finished
Processing feature3...   Finished
Processing feature4...   Finished
Building graph   Finished


In [61]:
print(dataset.edge_types)

[('user', 'following', 'user'), ('user', 'followedBy', 'user'), ('tweet', 'mentions', 'user'), ('tweet', 'retweets', 'user'), ('user', 'writes', 'tweet')]


In [62]:
from torch_geometric.loader import NeighborLoader

batch_size = 128
kwargs = {'num_workers': min(torch.cuda.device_count(),4) if torch.cuda.device_count() > 0 and torch.device.type == 'cuda' else 1, 'persistent_workers': True, 'batch_size': batch_size}
train_loader = NeighborLoader(dataset, num_neighbors=[100] * 3,shuffle=False, input_nodes=('user',dataset['user'].train_mask), **kwargs)

NeighborLoader()